# Taskmgr as Parent
Detects the creation of a process from Windows task manager

## Rule Content
```
- title: Taskmgr as Parent
  id: 3d7679bd-0c00-440c-97b0-3f204273e6c7
  status: experimental
  description: Detects the creation of a process from Windows task manager
  tags:
  - attack.defense_evasion
  - attack.t1036
  author: Florian Roth
  date: 2018/03/13
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      ParentImage: '*\taskmgr.exe'
    filter:
      Image:
      - '*\resmon.exe'
      - '*\mmc.exe'
      - '*\taskmgr.exe'
    condition: selection and not filter
  fields:
  - Image
  - CommandLine
  - ParentCommandLine
  falsepositives:
  - Administrative activity
  level: low

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_parent_path.keyword:*\\taskmgr.exe AND (NOT (process_path.keyword:(*\\resmon.exe OR *\\mmc.exe OR *\\taskmgr.exe))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()